# Python Intermediate

![python](https://upload.wikimedia.org/wikipedia/commons/c/c3/Python-logo-notext.svg)


_Don't you hate code that's not properly indented? Making it [indenting] part of the syntax guarantees that all code is properly indented._

--- Guido van Rossum

## Functional programming

![lambda](/files/images/lambda.png)

* A programming paradigm that treats computation as the evaluation of mathematical functions and avoids changing-state and mutable data. 
* It is a declarative programming paradigm, which means programming is done with expressions. 
* In functional code, the output value of a function depends only on the arguments that are input to the function, so calling a function f twice with the same value for an argument x will produce the same result f(x) each time. 

In [ ]:
# use function as parameter
def get_visit_ips(file_path, callback):
    with open(file_pth) as fp:
        return [callback(line) for line in fp]

In [ ]:
# return function
def cache(func):
    cached = {}
    def _func(attr, *args, **kwargs):
        if attr not in cached:
            cached[attr] = func(attr, *args, **kwargs)
        return cached[attr]
    
    return _func
        

In [ ]:
# functools.partial
import functools

def echo(name, city, country):
    print '%s live in %s, %s' % (name, city, country)
    
f = functools.partial(echo, city='Hangzhou', country='China')

f('Tom and Jerry')

In [ ]:
# lambda
lambda : True
lambda x: x ** 2
lambda x, y: x + y

In [ ]:
# map|reduce
map(lambda x: x ** 2, range(10))

reduce(lambda x, y: x + y, range(10))

### Practice

In [ ]:
# implement a to_int function, that convert hex string data to integer
# eg:
#     to_int('\xef')  ==> 239
#     to_int('\xef\x01')  ==> 61185
# NOTE: builtin function ord can return the integer ordinal of a one-character string


## Introspection

![introspection](/files/images/introspection.jpg)

In [ ]:
# dir
import os

print dir(os)

In [ ]:
# type
s = 'hello world'

print type(s)

In [ ]:
# isinstance & issubclass
class A(object):
    pass

class B(A):
    pass

a = A()
b = B()

print isinstance(a, A)
print isinstance(a, B)
print isinstance(b, B)
print isinstance(b, A)

print issubclass(B, A)
print issubclass(A, B)

In [ ]:
# hasattr
class DynamicObject(object):
    def __getattr__(self, attr):
        if attr in ('a', 'b', 'c'):
            return attr.upper()
        raise AttributeError

obj = DynamicObject()

print hasattr(obj, 'a')
print hasattr(obj, 'd')


In [ ]:
# inspect module
import inspect
import robot

print inspect.getdoc(robot)
print inspect.getsource(robot)
print inspect.getcomments(robot)

### Practice

In [ ]:
# 1. implement a bash wrapper, so that I can call bash command like a class attribute
#
#     bash = BashWrapper()
#     bash.ping('10.69.69.124')
#     bash.ls('-l', '~')
#
# Write your code here

# 2. what about a bash wrapper module, so that I can call bash command like a module import
#     from bashwrapper import ping
#     ping('10.69.69.124')
#     from bashwrapper import ls
#     ls('-l', '~')

## Unit Testing and TDD

In [ ]:
# Unit Testing
import unittest

def to_int(data):
    return 1

class TestToInt(unittest.TestCase):
    def test_to_int_with_one_char_string(self):
        self.assertEqual(to_int('\x01'), 1)
    
    def test_to_int_with_two_chars_string(self):
        self.assertEqual(to_int('\xef\x01'), 61185)

suite = unittest.TestLoader().loadTestsFromTestCase(TestToInt)
unittest.TextTestRunner().run(suite)

In [ ]:
# mock
import time

def delay_print(msg, delay):
    time.sleep(delay)
    print msg
    
import unittest

time.sleep = lambda x: True

class TestDelayPrint(unittest.TestCase):
        def test_delay_print_empty_string(self):
            delay_print('', 5)
            
suite = unittest.TestLoader().loadTestsFromTestCase(TestDelayPrint)
unittest.TextTestRunner().run(suite)

### TDD

![lambda](/files/images/tdd.jpg)

### Practice

Generate prime factors of a number

In [ ]:
# implement a score caculator for bowling game using TDD

## Iterator and Generator

![iterator](/files/images/iterator.png)

In [ ]:
# iterator
xrange(10)

fp = open('access_10000.log')
fp.next()
fp.close()

(e**2 for e in range(10))


In [ ]:
# generator
def series(length):
    for e in xrange(length):
        yield e ** 2

s = series(10)
print [e for e in s]

### Practice

In [ ]:
# implement a fibonacci series using iterator and generator

## Run in Parallel

![run](/files/images/run.png)

In [ ]:
# fetch content size from a series of web sites
import urllib

urls = ['http://hztdltev01.china.nsn-net.net', 'http://10.56.117.81/coci/', 'http://coop.china.nsn-net.net']

for url in urls:
    print len(urllib.urlopen(url, proxies={}).read())

In [ ]:
# introduce thread
from threading import Thread
import urllib

urls = ['http://hztdltev01.china.nsn-net.net', 'http://10.56.117.81/coci/', 'http://coop.china.nsn-net.net']

class UrlFetchThread(Thread):
    def __init__(self, url, *args):
        super(UrlFetchThread, self).__init__(*args)
        self._url = url
        
    def run(self):
        print len(urllib.urlopen(self._url).read())
        
threads = map(UrlFetchThread, urls)
for t in threads:
    t.start()
    t.join()
    

In [ ]:
# introduce multi process
from multiprocessing import Process
import urllib

urls = ['http://hztdltev01.china.nsn-net.net', 'http://10.56.117.81/coci/', 'http://coop.china.nsn-net.net']

class UrlFetchProcess(Process):
    def __init__(self, url, *args):
        super(UrlFetchProcess, self).__init__(*args)
        self._url = url
        
    def run(self):
        print len(urllib.urlopen(self._url).read())
        
processes = map(UrlFetchProcess, urls)
for p in processes:
    p.start()
    p.join()

In [ ]:
# use Pool
from multiprocessing import Pool
from multiprocessing.dummy import Pool as ThreadPool

urls = ['http://hztdltev01.china.nsn-net.net', 'http://10.56.117.81/coci/', 'http://coop.china.nsn-net.net']

def fetch_content(url):
    print len(urllib.urlopen(url).read())
    
pool = Pool()
pool.map(fetch_content, urls)
pool.close()
pool.join()

# -----------------------------------------
thread_pool = ThreadPool()
thread_pool.map(fetch_content, urls)
thread_pool.close()
thread_pool.join()

In [ ]:
# introduce gevent
import gevent
from gevent import monkey
monkey.patch_all()

urls = ['http://hztdltev01.china.nsn-net.net', 'http://10.56.117.81/coci/', 'http://coop.china.nsn-net.net']

def fetch_content(url):
    print len(urllib.urlopen(url).read())
    
[gevent.spawn(fetch_content, url) for url in urls]

gevent.wait()

### Practice

In [ ]:
# 1. implement a multi clients echo server
# 2. the echo server can support to execute command when the echo string startswith "!"
#      eg.
#          client              ->                     server
#           hello                                       hello
#           !ls                                            download   desktop  workspace
#  3. when the user send "quit", stop the echo server

## Web Development

![Python Web](/files/images/pyweb.png)

In [ ]:
# BaseHTTPServer and SimpleHTTPServer
# python -m SimpleHTTPServer 8080
from BaseHTTPServer import HTTPServer
from SimpleHTTPServer import SimpleHTTPRequestHandler

server = HTTPServer(('0.0.0.0', 8282), SimpleHTTPRequestHandler)

server.serve_forever()

#### Dive into BaseHTTPRequestHandler

* work process
    * setup
    * handle
    * finish
* do_GET
* do_POST
* ...

In [ ]:
# with micro framework bottle.py
from bottle import route, run, template

@route('/hello/<name>')
def index(name):
    return template('<b>Hello {{name}}</b>!', name=name)

run(host='localhost', port=8181)

## Use the database

In [ ]:
# sqlite3
import sqlite3 as sqlite

conn = sqlite.connect('user.db')
# create a table
conn.execute('create table user (id, name)')
# insert one record
conn.execute('insert into user values (123, "python")')
# query
cursor = conn.execute('select * from user')
try:
    print cursor.fetchall()
finally:
    cursor.close()

In [ ]:
# mongodb
import pymongo

# create connection
client = pymongo.MongoClient()
# select database
db = client.user
# select collection
collection = db.userinfo
# insert record
collection.insert_one({'id': 123, 'name': 'python'})
collection.insert_one({'id': 124, 'alias': 'python'})
# query
print [e for e in collection.find()]

## Reference

* http://www.diveintopython.net/power_of_introspection/
* https://docs.python.org/2/library/inspect.html
* https://docs.python.org/2/howto/functional.html
* https://en.wikipedia.org/wiki/Functional_programming
* https://docs.python.org/2/library/functions.html#iter
* http://butunclebob.com/files/downloads/Prime%20Factors%20Kata.ppt
* https://blog.8thlight.com/uncle-bob/2013/05/27/TheTransformationPriorityPremise.html
* https://wiki.python.org/moin/Generators
* https://docs.python.org/2/library/threading.html
* https://docs.python.org/2/library/multiprocessing.html
* http://www.gevent.org/intro.html
* http://bottlepy.org/docs/dev/index.html
* http://api.mongodb.org/python/current/tutorial.html
* https://docs.python.org/2/library/simplehttpserver.html

## Q & A